# Introduction

Enabling users to deploy models within their preferred environments, whether on local machines or Docker containers, Clarifai's `Inference Runners` offer a flexible solution. The Clarifai Python SDK further simplifies this process by providing a user-friendly interface for creating inference runners, remotely operating models, and initiating their service on the Clarifai Platform.


This notebook provides a step-by-step guide to the process of serving two models—one for image classification(ViT) and another for text generation(Phi2 LLM)—on a local machine using a Docker container.

### Getting Started

#### Obtaining a Clarifai PAT

To use the Clarifai Python SDK, you will need a PAT (Personal Access Token). You can obtain one by signing up for a free Clarifai account [here](https://clarifai.com/login). Once you have an account, you can find your PAT on the [Security](https://clarifai.com/settings/security) page.

Refer to the [documentation](https://docs.clarifai.com/clarifai-basics/authentication/personal-access-tokens) for more information on PATs.

#### Installing the Clarifai Python SDK
```bash
pip install clarifai
```

### Setup

In [ ]:
from clarifai.client.app import App
from clarifai.client.user import User
from clarifai.client.model import Model
from clarifai_grpc.grpc.api import resources_pb2
from google.protobuf.struct_pb2 import Struct

## Image Classification

### Create a Inference Runner

In [ ]:
client = User(user_id=USER_ID)
vit_classification_runner = client.create_runner("vit-cls-runner", labels=["cls-runner"], description="Vision Transformer Classification model from mmpretrain")

### Create a Remote Operator Model

In [ ]:
# Create a new app
app = client.create_app("runners-app")

In [ ]:
# Create remote operator model

params = Struct()
params.update({"runner_labels": ["cls-runner"]})
vit_remote_runner_model = app.create_model("vit-runner-model", model_type_id="remote-operator")
vit_remote_runner_model = vit_remote_runner_model.create_version(output_info=resources_pb2.OutputInfo(params=params))

### Start the remote operator VIT model server.

```bash
docker build -t vit-cls-runner -f Dockerfile-VIT .

docker run -e CLARIFAI_PAT=<YOUR_CLARIFAI_PAT> CLARIFAI_USER_ID=<YOUR_CLARIFAI_USER_ID> vit-cls-runner
```

### Prediction using the Remote Operator Model


In [ ]:
prediction_response = vit_remote_runner_model.predict_by_url("https://samples.clarifai.com/metro-north.jpg", "image")
print(prediction_response)

## Text Generation

### Create a Inference Runner

In [ ]:
client = User(user_id=USER_ID)
text_gen_runner = client.create_runner("sdk-phi2-runner", labels=["textgen-runner"], description="Text Generation Runner")

### Create a Remote Operator Model

In [ ]:
# Create remote operator model

app = client.app("runners-app")

params = Struct()
params.update({"runner_labels": ["textgen-runner"]})
phi2_remote_model = app.create_model("phi2-runner-model", model_type_id="remote-operator")
phi2_remote_model = phi2_remote_model.create_version(output_info=resources_pb2.OutputInfo(params=params))

### Start the remote operator Phi2 model server.

```bash
docker build -t text-gen-runner -f Dockerfile .

docker run -e CLARIFAI_PAT=<YOUR_CLARIFAI_PAT> CLARIFAI_USER_ID=<YOUR_CLARIFAI_USER_ID> text-gen-runner
```

### Prediction using the Remote Operator Model

In [ ]:
prediction_response = phi2_remote_model.predict_by_bytes(b"Explain about Photosynthesis ?", "text")
print(prediction_response)

## Clarifai Resources

**Website**: [https://www.clarifai.com](https://www.clarifai.com/)

**Demo**: [https://clarifai.com/demo](https://clarifai.com/demo)

**Sign up for a free Account**: [https://clarifai.com/signup](https://clarifai.com/signup)

**Developer Guide**: [https://docs.clarifai.com](https://docs.clarifai.com/)

**Clarifai Community**: [https://clarifai.com/explore](https://clarifai.com/explore)

**Python SDK Docs**: [https://docs.clarifai.com/python-sdk/api-reference](https://docs.clarifai.com/python-sdk/api-reference)

---